In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from pixelmed_calc.medical_imaging.modelstore import get_model_from_name
from pixelmed_calc.medical_imaging.DLtools.process import load_data_from_csv,get_file_and_labels,get_transforms,train_and_validate
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from monai.data import ImageDataset
import os
import pandas as pd
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F


# CSV文件路径
csv_path = r'E:\127lungnode\lungnode\group.csv'
train_base_path = r'E:\127lungnode\lungnode\crop3d'
val_base_path = r'E:\127lungnode\lungnode\crop3d'

tfs=get_transforms(resize=[16,128,128])
# 读取所有数据
file_names, labels, groups = load_data_from_csv(csv_path)
# 获取训练数据和标签（可以设置 base_path 为 None 或实际路径）
file_train, train_labels = get_file_and_labels(file_names, labels, groups, 'train', train_base_path)
# 获取验证数据和标签（可以设置 base_path 为 None 或实际路径）
file_val, val_labels = get_file_and_labels(file_names, labels, groups, 'val', val_base_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = get_model_from_name['resnet50_3D'](num_classes=2, pretrained=False, in_channels=1).to(device)
model=get_model_from_name['vit_3D'](
    image_size = 128,          # image size
    frames = 16,               # number of frames
    image_patch_size = 16,     # image patch size
    frame_patch_size = 2,      # frame patch size
    num_classes = 2,
    channels=1,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
).to("cuda")
model.load_state_dict(torch.load(r'd:\pixelmedAI\function_pm\note2-深度学习分类\results\latest_model.pth'))

<class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.


<All keys matched successfully>

In [2]:
model

ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (f pf) (h p1) (w p2) -> b (f h w) (p1 p2 pf c)', p1=16, p2=16, pf=2)
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=512, out_features=1024, bias=True)
    (3): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0-5): 6 x ModuleList(
        (0): Attention(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attend): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (to_qkv): Linear(in_features=1024, out_features=1536, bias=False)
          (to_out): Sequential(
            (0): Linear(in_features=512, out_features=1024, bias=True)
            (1): Dropout(p=0.1, inplace=False)
          )
        )
        (1): FeedForward(
          (net): Sequential(
            (0): LayerNorm((1024,), eps=1e-05, elementwis

In [5]:
import numpy as np
import pandas as pd
from pixelmed_calc.medical_imaging.DLtools.utils import extract_layer_output
val_ds_viz = ImageDataset(image_files=file_train, transform=tfs)  
val_loader_viz = DataLoader(val_ds_viz, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())  

DL_output=pd.DataFrame()
i=0
for sample in val_loader_viz:  
        ids = val_loader_viz.dataset.image_files[i]
        avg_pool_output = extract_layer_output(model, sample, 'to_latent')  
        avg_pool_output=avg_pool_output.cpu().detach().numpy()
        avg_pool_output=avg_pool_output.reshape(1,-1)
        ids_df = pd.Series(ids)  
        avg_pool_output_df = pd.DataFrame(avg_pool_output)  
        # Concatenate ids and avg_pool_output horizontally  
        combined_df = pd.concat([ids_df.reset_index(drop=True), avg_pool_output_df.reset_index(drop=True)], axis=1)  
        # Append the combined DataFrame to DL_output  
        DL_output = pd.concat([DL_output, combined_df], ignore_index=True) 
        i=i+1

DL_output


,0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-3.219762,-1.849828,-1.637610,1.092938,0.632654,1.071192,0.307791,-0.209582,-3.171879,...,2.003549,-0.603940,0.221340,0.136147,-0.789182,1.522895,-1.659826,1.422286,-1.155413,0.673044
1,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-3.550760,0.555781,-1.359864,1.220370,0.249610,0.903174,2.885754,0.021393,-3.898000,...,2.099704,-0.102745,0.310928,-2.279068,-0.756688,1.535906,-2.176202,0.009983,-4.284792,1.223273
2,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-2.877797,-1.586038,-2.692952,-0.059075,-0.359158,0.163649,3.259487,0.048063,-4.345622,...,2.789541,-0.074575,0.144673,-3.009571,-0.905605,1.816200,-1.309780,1.802627,-3.602583,1.062746
3,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-3.139283,-1.314989,-1.790961,1.112546,0.330587,0.987592,3.245713,-0.732413,-3.440111,...,2.837510,-0.203025,-0.643648,-1.935381,-1.069758,1.314194,0.003793,1.269376,-3.738696,0.709486
4,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-2.331195,-3.104111,1.079375,2.011683,0.770958,1.985214,1.651312,-0.112428,-2.332543,...,2.475876,1.181092,0.861683,-0.902651,1.153287,2.192657,-3.249985,0.911340,-4.999857,-0.293487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-2.972063,0.469674,-1.766090,1.295601,0.394052,0.926048,2.799592,-1.004801,-3.086268,...,1.063035,0.359166,0.735883,-1.868870,-1.225538,0.881016,-2.966277,1.608686,-3.518197,1.235360
335,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-2.491507,-2.456388,1.023940,2.288928,-0.607518,2.279867,1.654242,-0.535944,-1.975640,...,2.765795,1.439398,1.180233,-0.979021,0.824530,1.098710,-4.098340,0.796945,-5.527397,-1.167979
336,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-3.895252,-0.316812,-2.987416,0.778099,-0.017841,0.658743,2.468706,-0.613599,-4.687973,...,2.833319,0.695789,0.215893,-1.636880,-1.164237,2.150215,-1.549124,1.680851,-2.771079,1.974180
337,E:\127lungnode\lungnode\crop3d\patient2_1.nii.gz,-3.684372,-1.630344,-1.405397,1.460185,0.385745,0.750918,2.777408,-0.466096,-4.304763,...,2.199464,-0.072136,0.546786,-1.492080,-1.273627,1.190289,-0.900676,1.267254,-3.467908,1.022539


In [6]:
DL_output.to_csv('DL_feature.csv',index=False)